In [1]:
import sys
sys.path.append('/home/UR/kontaous/Bureau/Stage_Data/')
from Analyses.traitement import *
import pandas as pd
import numpy as np
from Analyses.traitement import *
tracking = pd.read_csv("../../../Tracking_data/MontPellier.csv")
events = pd.read_csv("../../Data/match_data/match_Montpellier.csv")
tracking, events = prepare_data(tracking, events)
tracking = tracking.sort_values(by="system_time")
tracking.set_index('frame', inplace=True)
tracking.drop(['Gx', 'Gy', 'id', 'num', 'player'], axis=1, inplace=True)
tracking.rename(columns={'player_name': 'player.name', 'team': 'team.name', 'x': 'location.x', 'y': 'location.y', }, inplace=True)
data_tracking = "../../../Tracking_data/new_MontPellier.csv"
sauvegarder_dataframe_en_csv(tracking, data_tracking)

DataFrame sauvegardé en tant que ../../../Tracking_data/new_MontPellier.csv


In [2]:
df_passes = filtrer_et_formater_passes(events)
data_passes = "../../Data/Tracking/montpellier_passes.csv"
sauvegarder_dataframe_en_csv(df_passes, data_passes)

DataFrame sauvegardé en tant que ../../Data/Tracking/montpellier_passes.csv


In [6]:
df_passes = pd.read_csv('../../Data/Tracking/montpellier_passes.csv')

In [ ]:
import pandas as pd
import plotly.graph_objs as go

def visualize_pass_network(player_positions_df, passes_df, period, team):
    # Filtrer les passes pour la période et l'équipe spécifiées
    passes_df = passes_df[(passes_df['period'] == period) & (passes_df['team.name'] == team)]
    player_positions_df = player_positions_df[player_positions_df['team.name'] == team]

    # Convertir les dataframes en dictionnaires
    players_positions = {
        row['player.name']: (row['location.x'], row['location.y'])
        for index, row in player_positions_df.iterrows()
    }

    # Calculer les informations des passes
    pass_info = passes_df.groupby(['player.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    pass_info_received = passes_df.groupby(['pass.recipient.name']).agg(
        num_passes_received=('pass.length', 'size'),
        avg_pass_length_received=('pass.length', 'mean')
    ).reset_index()

    # Créer des dictionnaires pour les infos de passes
    pass_info_dict = {
        row['player.name']: {
            'num_passes': row['num_passes'],
            'avg_pass_length': row['avg_pass_length']
        }
        for index, row in pass_info.iterrows()
    }

    pass_info_received_dict = {
        row['pass.recipient.name']: {
            'num_passes_received': row['num_passes_received'],
            'avg_pass_length_received': row['avg_pass_length_received']
        }
        for index, row in pass_info_received.iterrows()
    }

    # Tracer le terrain de football
    fig = go.Figure()

    # Ajout des formes pour représenter le terrain de football
    terrain_shapes = [
        dict(type="rect", x0=0, y0=0, x1=120, y1=80, line=dict(color="Black")),
        dict(type="line", x0=60, y0=0, x1=60, y1=80, line=dict(color="Black", dash="dot")),
        dict(type="rect", x0=0, y0=18, x1=18, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=102, y0=18, x1=120, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=30, x1=5.5, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=114.5, y0=30, x1=120, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=36, x1=2, y1=44, line=dict(color="Black")),
        dict(type="rect", x0=118, y0=36, x1=120, y1=44, line=dict(color="Black")),
        dict(type="circle", xref="x", yref="y", x0=50.4, y0=30.4, x1=69.6, y1=49.6, line_color="Black"),
    ]
    fig.update_layout(shapes=terrain_shapes)

    # Ajout des joueurs sur le terrain avec les infos de passes
    for player, pos in players_positions.items():
        passes_from_player = pass_info_dict.get(player, {'num_passes': 0, 'avg_pass_length': 0})
        passes_to_player = pass_info_received_dict.get(player, {'num_passes_received': 0, 'avg_pass_length_received': 0})

        hover_text = (
            f"Joueur: {player}<br>"
            f"Nombre de passes effectuées: {passes_from_player['num_passes']}<br>"
            f"Longueur moyenne des passes effectuées: {passes_from_player['avg_pass_length']:.2f} m<br>"
            f"Nombre de passes reçues: {passes_to_player['num_passes_received']}<br>"
            f"Longueur moyenne des passes reçues: {passes_to_player['avg_pass_length_received']:.2f} m"
        )

        fig.add_trace(go.Scatter(
            x=[pos[0]],
            y=[pos[1]],
            mode='markers+text',
            text=[player],
            textposition="top center",
            marker=dict(size=10, color='blue'),
            hovertext=hover_text,
            hoverinfo="text"
        ))

    # Ajout des passes
    for passe in passes_df.itertuples():
        if passe._4 in players_positions and passe._5 in players_positions:
            start = players_positions[passe._4]
            end = players_positions[passe._5]
            fig.add_trace(go.Scatter(
                x=[start[0], end[0]],
                y=[start[1], end[1]],
                mode='lines',
                line=dict(width=2, color='red')
            ))

    fig.update_layout(
        title=f"Réseau de passes sur un terrain de football (Période {period}, Équipe {team})",
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False),
        width=800,
        height=600
    )
    fig.show()

visualize_pass_network(df_positions, df_passes, 1, "Montpellier")


In [ ]:
import pandas as pd
import plotly.graph_objs as go


roles_positions = {
    "Gardien": (3, 40),
    "Défenseur": (12, 30),
    "Défenseur": (12, 50),
    "Défenseur ": (25, 10),
    "Défenseur": (25, 70),
    "Milieu ": (50, 40),
    "Milieu": (80, 20),
    "Milieu": (80, 60),
    "Ailier ": (100, 10),
    "Ailier": (100, 70),
    "Attaquant": (110, 40)
}

def assign_positions(player_positions_df):
    # Assigner les positions aux joueurs en fonction de leur rôle
    player_positions_df['location.x'] = player_positions_df['role'].apply(lambda x: roles_positions.get(x, (0, 0))[0])
    player_positions_df['location.y'] = player_positions_df['role'].apply(lambda x: roles_positions.get(x, (0, 0))[1])
    return player_positions_df

def visualize_pass_network(player_positions_df, passes_df, period, team):
    # Assigner les positions aux joueurs
    player_positions_df = assign_positions(player_positions_df)

    # Filtrer les passes pour la période et l'équipe spécifiées
    passes_df = passes_df[(passes_df['period'] == period) & (passes_df['team.name'] == team)]
    player_positions_df = player_positions_df[player_positions_df['team.name'] == team]

    # Convertir les dataframes en dictionnaires
    players_positions = {
        row['player.name']: (row['location.x'], row['location.y'])
        for index, row in player_positions_df.iterrows()
    }

    # Calculer les informations des passes
    pass_info = passes_df.groupby(['player.name']).agg(
        num_passes=('pass.length', 'size'),
        avg_pass_length=('pass.length', 'mean')
    ).reset_index()

    pass_info_received = passes_df.groupby(['pass.recipient.name']).agg(
        num_passes_received=('pass.length', 'size'),
        avg_pass_length_received=('pass.length', 'mean')
    ).reset_index()

    # Créer des dictionnaires pour les infos de passes
    pass_info_dict = {
        row['player.name']: {
            'num_passes': row['num_passes'],
            'avg_pass_length': row['avg_pass_length']
        }
        for index, row in pass_info.iterrows()
    }

    pass_info_received_dict = {
        row['pass.recipient.name']: {
            'num_passes_received': row['num_passes_received'],
            'avg_pass_length_received': row['avg_pass_length_received']
        }
        for index, row in pass_info_received.iterrows()
    }

    # Tracer le terrain de football
    fig = go.Figure()

    # Ajout des formes pour représenter le terrain de football
    terrain_shapes = [
        dict(type="rect", x0=0, y0=0, x1=120, y1=80, line=dict(color="Black")),
        dict(type="line", x0=60, y0=0, x1=60, y1=80, line=dict(color="Black", dash="dot")),
        dict(type="rect", x0=0, y0=18, x1=18, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=102, y0=18, x1=120, y1=62, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=30, x1=5.5, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=114.5, y0=30, x1=120, y1=50, line=dict(color="Black")),
        dict(type="rect", x0=0, y0=36, x1=2, y1=44, line=dict(color="Black")),
        dict(type="rect", x0=118, y0=36, x1=120, y1=44, line=dict(color="Black")),
        dict(type="circle", xref="x", yref="y", x0=50.4, y0=30.4, x1=69.6, y1=49.6, line_color="Black"),
    ]
    fig.update_layout(shapes=terrain_shapes)

    # Ajout des joueurs sur le terrain avec les infos de passes
    for player, pos in players_positions.items():
        passes_from_player = pass_info_dict.get(player, {'num_passes': 0, 'avg_pass_length': 0})
        passes_to_player = pass_info_received_dict.get(player, {'num_passes_received': 0, 'avg_pass_length_received': 0})

        hover_text = (
            f"Joueur: {player}<br>"
            f"Nombre de passes effectuées: {passes_from_player['num_passes']}<br>"
            f"Longueur moyenne des passes effectuées: {passes_from_player['avg_pass_length']:.2f} m<br>"
            f"Nombre de passes reçues: {passes_to_player['num_passes_received']}<br>"
            f"Longueur moyenne des passes reçues: {passes_to_player['avg_pass_length_received']:.2f} m"
        )

        fig.add_trace(go.Scatter(
            x=[pos[0]],
            y=[pos[1]],
            mode='markers+text',
            text=[player],
            textposition="top center",
            marker=dict(size=10, color='blue'),
            hovertext=hover_text,
            hoverinfo="text"
        ))

    # Ajout des passes
    for passe in passes_df.itertuples():
        if passe._4 in players_positions and passe._5 in players_positions:
            start = players_positions[passe._4]
            end = players_positions[passe._5]
            fig.add_trace(go.Scatter(
                x=[start[0], end[0]],
                y=[start[1], end[1]],
                mode='lines',
                line=dict(width=2, color='red')
            ))

    fig.update_layout(
        title=f"Réseau de passes sur un terrain de football (Période {period}, Équipe {team})",
        xaxis=dict(range=[0, 120], showgrid=False, zeroline=False),
        yaxis=dict(range=[0, 80], showgrid=False, zeroline=False),
        width=800,
        height=600
    )
    fig.show()



visualize_pass_network(df_positions, df_passes, 1, "Montpellier")
